# Cleaning Data Using Pandas

In this activity, we will demonstrate how you can clean the data in Python using appropriate Python functions and libraries. 

Python is a great language for doing data analysis, primarily because of the fantastic ecosystem of data-centric python packages. Pandas is one of those packages and makes importing and analyzing data much easier.
An important part of Data analysis is analyzing Duplicate Values and removing them. Pandas drop_duplicates() method helps in removing duplicates from the data frame.


## Data Cleaning: Null values

- Sometimes values will be blank in your data
- It could be an error; it could be the data was not available
- There are some techniques to deal with it; all of them are imperfect
- These techniques should be employed only if you can't get better data


## Reading a CSV file

- Dataset: https://www.kaggle.com/uciml/pima-indians-diabetes-database/data#

In [1]:
import pandas as pd
import numpy as np

diabetes_df = pd.read_csv('diabetes.csv')
diabetes_df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


### Null values: Summary of the data

- Sometimes null values aren't exactly NaNs
- They are encoded as -1 or 9999 etc.
- Sometimes it's 0. 
- Does 0 make sense for some of these categories??

A good function to use to quickly find out how data in each column is distributed is `pd.describe()`. This function provides descriptive statistics, including those that summarize the central tendency, dispersion and shape of a dataset’s distribution. When providing these descriptive statistics, it does so by excluding NaN values.

The `pd.describe()` function can analyzes both numeric and object series, as well as DataFrame column sets of mixed data types. There is an optional parametr `include`. When `incldue="all"`, the output also include columns that are not nuemrical.

When null values aren't showing up aas NaNs but rather coded as some other numebr, using `pd.describe()` can help us locate that. For example, the following code block tells us that the minimum value for the Glucose column is 0. That doesn't make sense. Similarly, Blood Pressure, Skin Thickness, Insulin level, and BMI columns all have minimum values of 0. 

In [2]:
diabetes_df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


### Figuring out where the NaNs are (this only works if the data is actually NaN)

In [3]:
diabetes_df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

### Null values: Encoding true NaNs as NaNs

The null value shouldn't be used in summary calculations (e.g., average, count), so it is important to identify if missing values are somehow coded as some extreme numerical value.

Some columns have a lot of what we think could be missing values, and it is important to identify what those columns are.

In [4]:
diabetes_df['Glucose']==0

0      False
1      False
2      False
3      False
4      False
       ...  
763    False
764    False
765    False
766    False
767    False
Name: Glucose, Length: 768, dtype: bool

We can pass in a list of column names (`cols_missing_vals`), and use it to find out how many missing values there are in each of these columns.

To Select multiple columns, put a column names inside a Python list, and put that list inside a pair of square brackets.

In [5]:
cols_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI'] # cols with missing values
(diabetes_df[cols_missing_vals] == 0).sum() # count number of 0s

Glucose            5
BloodPressure     35
SkinThickness    227
Insulin          374
BMI               11
dtype: int64

At this point, we can fill the 0s in these columns as `np.NaN`. This will code the missing value to be NaN. 

Because these values are now NaN, when we compute summary statistics, they won't be factored into the calculation, since they are no longer a number. If this step was not done, the 0s would impact the summary statistics. 

In [6]:
diabetes_df[cols_missing_vals] = diabetes_df[cols_missing_vals].replace(0, np.NaN) # replace 0's with NaNs
diabetes_df.isnull().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

### Null values: Removing rows with missing values
- Could be a good idea if there aren't too many records removed
- Let's do this for Glucose and BMI columns
- documentation for pandas dropna() function: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
- `pd.dropna()` has a default parameter, how='any'. Possible input values are 'any', or 'all'

In [7]:
# pd.dropna() has a default parameter, how='any'. Possible input values are 'any', or 'all'

print("Shape before dropping NAs", diabetes_df.shape)

diabetes_df = diabetes_df.dropna(subset=['Glucose', 'BMI']) # drop rows with Glucose and BMI as NaN

print("Shape after dropping NAs for Glucose and BMI columns", diabetes_df.shape)

Shape before dropping NAs (768, 9)
Shape after dropping NAs for Glucose and BMI columns (752, 9)


### Null values: using the average

In [8]:
# Fill in missing values with the average
diabetes_df['SkinThickness'] = diabetes_df['SkinThickness'].fillna(value=diabetes_df['SkinThickness'].mean())
diabetes_df.isnull().sum()

C:\Users\eddie\AppData\Local\Temp\ipykernel_17716\289822400.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diabetes_df['SkinThickness'] = diabetes_df['SkinThickness'].fillna(value=diabetes_df['SkinThickness'].mean())


Pregnancies                   0
Glucose                       0
BloodPressure                28
SkinThickness                 0
Insulin                     360
BMI                           0
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

## Data Cleaning: Duplicate Rows

Pandas provides a good function to clean data from duplicate values: `pd.drop_duplicates()`. 

The `pd.drop_duplicates()` function has a few paramters that you can specify values:

`subset`: Subset takes a column or list of column label. It’s default value is none. After passing columns, it will consider them only for duplicates. 
`keep`: keep is to control how to consider duplicate value. It has only three distinct value and default is ‘first’. 

- If ‘first’, it considers first value as unique and rest of the same values as duplicate.
- If ‘last’, it considers last value as unique and rest of the same values as duplicate.
- If False, it consider all of the same values as duplicates

`inplace`: Boolean values, removes rows with duplicates if True.

It then returns a DataFrame with removed duplicate rows depending on Arguments passed. 

### Removing rows with the same value in a column

In [9]:
 
# making data frame from csv file
data = pd.read_csv("employees.csv")
 
# sorting by first name
data.sort_values("First Name", inplace = True)
 
# dropping ALL duplicate values
data.drop_duplicates(subset ="First Name",
                     keep = False, inplace = True)
 
# displaying data
data

,Emp ID,First Name,Last Name,Gender,E Mail,Date of Birth,Date of Joining,Salary,SSN,Phone No.,County,City,State,Zip
4905,859085,Aaron,Laplante,M,aaron.laplante@hotmail.com,3/31/1978,6/24/2016,97879,536-71-5525,218-588-2428,Fillmore,Spring Valley,MN,55975
2621,775215,Abel,Varner,M,abel.varner@yahoo.com,5/1/1974,12/4/1997,187021,715-18-4802,701-817-8263,Bottineau,Antler,ND,58711
1588,967670,Adah,Hofmann,F,adah.hofmann@hotmail.com,3/15/1977,8/16/2001,101648,597-92-2979,262-906-6238,Brown,Green Bay,WI,54313
763,445332,Adaline,Byrnes,F,adaline.byrnes@charter.net,1/5/1990,2/22/2011,143179,731-28-0283,231-910-0633,Genesee,Burton,MI,48529
3516,466526,Adan,Wesley,M,adan.wesley@gmail.com,6/3/1970,8/28/2001,104606,171-86-9901,217-799-8172,Randolph,Walsh,IL,62297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,384939,Zenaida,Cavazos,F,zenaida.cavazos@gmail.com,4/27/1965,4/8/2000,95950,417-67-5948,229-925-1431,Monroe,Bolingbroke,GA,31004
4784,578209,Zora,Maupin,F,zora.maupin@aol.com,12/25/1957,7/16/1988,188869,257-99-2566,505-935-9867,San Miguel,Ribera,NM,87560
4348,150828,Zoraida,Sanchez,F,zoraida.sanchez@charter.net,3/29/1968,12/9/2015,126843,346-08-5092,212-305-3599,Delaware,Meridale,NY,13806
668,772602,Zula,Romeo,F,zula.romeo@verizon.net,1/18/1966,7/17/2011,168294,065-02-2599,212-317-3624,Onondaga,Syracuse,NY,13251


### Removing rows with all duplicate values 

In [10]:
data = pd.read_csv("employees.csv")
length1 = len(data)
 
# manually inserting duplicate of a row of row 440
data.loc[1001] = [data["Emp ID"][440],
                  data["First Name"][440],
                  data["Last Name"][440],
                  data["Gender"][440],
                  data["E Mail"][440],
                  data["Date of Birth"][440],
                  data["Date of Joining"][440],
                  data["Salary"][440],
                  data["SSN"][440],
                  data["Phone No. "][440],
                  data["County"][440],
                  data["City"][440],
                  data["State"][440],
                  data["Zip"][440]]
                   

# length after adding row
length2=  len(data)
 
# sorting by first name
data.sort_values("First Name", inplace=True)
 
# dropping duplicate values
data.drop_duplicates(keep=False,inplace=True)
 

## Data Cleaning: Wrong Format

The detailed walkthrough is covered in the Formatting Data activity.